## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from json import dumps
pd.set_option(
    'display.max_colwidth', 100
)


## Загрузка данных и первичный анализ

In [4]:
try:
    df = pd.read_json("/Users/Svetlana/Desktop/trial_task.json")
except FileNotFoundError:
    df = pd.read_json(
        "https://drive.google.com/file/d/1mfTIy7zW3G7yrbr-VhnOtkFeT1UMdu0l/view?usp=sharing"
    )

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        100 non-null    int64 
 1   warehouse_name  100 non-null    object
 2   highway_cost    100 non-null    int64 
 3   products        100 non-null    object
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


In [6]:
df.sample(10)

,order_id,warehouse_name,highway_cost,products
61,22103,гиперборея,-60,"[{'product': 'плюмбус', 'price': 250, 'quantity': 1}, {'product': 'подписка на suppi-блог', 'pri..."
28,70074,остров невезения,-30,"[{'product': 'плюмбус', 'price': 250, 'quantity': 2}, {'product': 'плюмбус', 'price': 250, 'quan..."
85,33780,гиперборея,-120,"[{'product': 'подписка на suppi-блог', 'price': 150, 'quantity': 2}, {'product': 'статуэтка Лени..."
31,32313,хутор близ Диканьки,-120,"[{'product': 'билет в Израиль', 'price': 1000, 'quantity': 3}, {'product': 'плюмбус', 'price': 2..."
94,36764,гиперборея,-20,"[{'product': 'подписка на suppi-блог', 'price': 150, 'quantity': 1}]"
19,83474,хутор близ Диканьки,-75,"[{'product': 'плюмбус', 'price': 250, 'quantity': 2}, {'product': 'ломтик июльского неба', 'pric..."
71,72309,остров невезения,-5,"[{'product': 'билет в Израиль', 'price': 1000, 'quantity': 1}]"
82,99246,хутор близ Диканьки,-45,"[{'product': 'автограф Стаса Барецкого', 'price': 600, 'quantity': 3}]"
25,68707,отель Лето,-50,"[{'product': 'плюмбус', 'price': 250, 'quantity': 2}]"
97,79293,отель Лето,-75,"[{'product': 'статуэтка Ленина', 'price': 200, 'quantity': 1}, {'product': 'автограф Стаса Барец..."


In [7]:
# проверим пропуски
df.isna().sum()

order_id          0
warehouse_name    0
highway_cost      0
products          0
dtype: int64

In [8]:
# проверяем столбец id на уникальные номера
if df["order_id"].nunique() == len(df):
    print("Столбец order_id содержит уникальные номера")
else:
    print("Столбец order_id содержит повторяющиеся номера")

Столбец order_id содержит уникальные номера


In [9]:
# в данных о стоимости доставки много отрицательных значений. Заменим на абсолютные значения. Проверим
df["highway_cost"] = df["highway_cost"].apply(abs)

In [10]:
df.head(5)

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,70,"[{'product': 'ломтик июльского неба', 'price': 450, 'quantity': 1}, {'product': 'билет в Израиль..."
1,62239,хутор близ Диканьки,15,"[{'product': 'билет в Израиль', 'price': 1000, 'quantity': 1}]"
2,85794,отель Лето,50,"[{'product': 'зеленая пластинка', 'price': 10, 'quantity': 2}]"
3,33684,Мордор,30,"[{'product': 'билет в Израиль', 'price': 1000, 'quantity': 2}, {'product': 'зеленая пластинка', ..."
4,25824,отель Лето,75,"[{'product': 'автограф Стаса Барецкого', 'price': 600, 'quantity': 1}, {'product': 'статуэтка Ле..."


In [11]:
# Колонка products представлена списком словарей. Информацию из этих словарей добавим в таблицу.
df = pd.concat(
    [df.drop(["products"], axis=1), df["products"].apply(lambda x: pd.Series(x[0]))],
    axis=1,
)

In [12]:
df.head(10)

,order_id,warehouse_name,highway_cost,product,price,quantity
0,11973,Мордор,70,ломтик июльского неба,450,1
1,62239,хутор близ Диканьки,15,билет в Израиль,1000,1
2,85794,отель Лето,50,зеленая пластинка,10,2
3,33684,Мордор,30,билет в Израиль,1000,2
4,25824,отель Лето,75,автограф Стаса Барецкого,600,1
5,87044,остров невезения,15,плюмбус,250,3
6,58598,гиперборея,160,плюмбус,250,3
7,5430,гиперборея,80,ломтик июльского неба,450,1
8,60502,отель Лето,75,автограф Стаса Барецкого,600,3
9,96473,Мордор,20,зеленая пластинка,10,2


### 1. Расчет тарифа стоимости доставки

По условию задачи, когда заказ доставляется из склада, списывается стоимость доставки. У каждого склада есть определенный тариф, определяющий стоимость доставки, не завсисимо от того, какой именно товар доставляется. Это тариф имеет размерность стоимость доставки на единицу товара.

В таком случае, Тариф = Сумма за доставку / Количество отправок с каждого склада


In [13]:
df["delivery_rate"] = df["highway_cost"] / df["quantity"]

In [15]:
tariff = df.groupby("warehouse_name").agg({"highway_cost": "sum", "quantity": "count"})
tariff["delivery_rate"] = tariff["highway_cost"] / tariff["quantity"]
tariff = tariff.reset_index()

In [16]:
display(tariff)

,warehouse_name,highway_cost,quantity,delivery_rate
0,Мордор,510,15,34.000000
1,гиперборея,1720,21,81.904762
2,остров невезения,280,16,17.500000
3,отель Лето,1600,20,80.000000
4,хутор близ Диканьки,1770,28,63.214286


### 2. Расчет суммарного количества, суммарного дохода, суммарного расхода и суммарной прибыли для каждого товара (результаты представлены в таблице)

По условию задания, 

доходом с товара является цена продажи * количество товара

расходом является тариф для данного склада * количество товара

прибылью является доход - расход

In [17]:
df["income"] = df["price"] * df["quantity"]

In [18]:
df["expense"] = df["delivery_rate"] * df["quantity"]

In [19]:
df["profit"] = df["income"] - df["expense"]

In [20]:
product_df = (
    df.groupby("product")
    .agg({"quantity": "sum", "income": "sum", "expense": "sum", "profit": "sum"})
    .reset_index()
)

In [21]:
display(product_df)

,product,quantity,income,expense,profit
0,автограф Стаса Барецкого,23,13800,815.0,12985.0
1,билет в Израиль,31,31000,965.0,30035.0
2,зеленая пластинка,31,310,1120.0,-810.0
3,ломтик июльского неба,21,9450,575.0,8875.0
4,плюмбус,34,8500,1080.0,7420.0
5,подписка на suppi-блог,16,2400,715.0,1685.0
6,статуэтка Ленина,25,5000,610.0,4390.0


### 3. Составление таблицы со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). Расчет средней прибыли заказов.

In [22]:
df_order_profit = df.groupby("order_id")["profit"].sum().reset_index()

In [23]:
df_order_profit

,order_id,profit
0,124,705.0
1,1391,490.0
2,2091,800.0
3,2108,0.0
4,2558,175.0
...,...,...
95,98100,590.0
96,98423,1170.0
97,99220,675.0
98,99246,1755.0


In [24]:
average_profit = df_order_profit["profit"].mean()

In [25]:
print("средняя прибыль заказов", average_profit)

средняя прибыль заказов 645.8


### 4. Составление таблицы со столбцами 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада).

In [35]:
merged_df = df.merge(df_order_profit, on="order_id")

In [27]:
merged_df = merged_df.drop("profit_y", axis=1)
merged_df = merged_df.rename(columns={"profit_x": "profit"})
merged_df

,order_id,warehouse_name,highway_cost,product,price,quantity,delivery_rate,income,expense,profit
0,11973,Мордор,70,ломтик июльского неба,450,1,70.0,450,70.0,380.0
1,62239,хутор близ Диканьки,15,билет в Израиль,1000,1,15.0,1000,15.0,985.0
2,85794,отель Лето,50,зеленая пластинка,10,2,25.0,20,50.0,-30.0
3,33684,Мордор,30,билет в Израиль,1000,2,15.0,2000,30.0,1970.0
4,25824,отель Лето,75,автограф Стаса Барецкого,600,1,75.0,600,75.0,525.0
...,...,...,...,...,...,...,...,...,...,...
95,28106,хутор близ Диканьки,30,подписка на suppi-блог,150,1,30.0,150,30.0,120.0
96,98100,остров невезения,10,автограф Стаса Барецкого,600,1,10.0,600,10.0,590.0
97,79293,отель Лето,75,статуэтка Ленина,200,1,75.0,200,75.0,125.0
98,2930,Мордор,30,плюмбус,250,2,15.0,500,30.0,470.0


In [28]:
grouped_df = merged_df.groupby(["warehouse_name", "product"]).agg(
    {"profit": "sum", "quantity": "sum"}
)

grouped_df["percent_profit_product_of_warehouse"] = (
    grouped_df["profit"]
    / grouped_df.groupby("warehouse_name")["profit"].transform("sum")
    * 100
)

final_df = grouped_df[["quantity", "profit", "percent_profit_product_of_warehouse"]]

In [29]:
final_df

quantity  profit  \
warehouse_name      product                                      
Мордор              автограф Стаса Барецкого         1   570.0   
                    билет в Израиль                  6  5930.0   
                    зеленая пластинка               10   -60.0   
                    ломтик июльского неба            3  1260.0   
                    плюмбус                          2   470.0   
                    подписка на suppi-блог           2   220.0   
                    статуэтка Ленина                 2   350.0   
гиперборея          автограф Стаса Барецкого         5  2760.0   
                    билет в Израиль                  8  7780.0   
                    зеленая пластинка                7  -310.0   
                    ломтик июльского неба            2   800.0   
                    плюмбус                          6  1140.0   
                    подписка на suppi-блог           9   930.0   
остров невезения    автограф Стаса Барецкого         2  1180.0   
                    билет в Израиль                  1   995.0   
                    зеленая пластинка                4   -15.0   
                    ломтик июльского неба            5  2210.0   
                    плюмбус                          8  1915.0   
                    статуэтка Ленина                10  1925.0   
отель Лето          автограф Стаса Барецкого         4  2250.0   
                    билет в Израиль                 10  9525.0   
                    зеленая пластинка                6  -315.0   
                    ломтик июльского неба            8  3300.0   
                    плюмбус                          2   450.0   
                    подписка на suppi-блог           2   175.0   
                    статуэтка Ленина                 3   475.0   
хутор близ Диканьки автограф Стаса Барецкого        11  6225.0   
                    билет в Израиль                  6  5805.0   
                    зеленая пластинка                4  -110.0   
                    ломтик июльского неба            3  1305.0   
                    плюмбус                         16  3445.0   
                    подписка на suppi-блог           3   360.0   
                    статуэтка Ленина                10  1640.0   

                                              percent_profit_product_of_warehouse  
warehouse_name      product                                                        
Мордор              автограф Стаса Барецкого                             6.521739  
                    билет в Израиль                                     67.848970  
                    зеленая пластинка                                   -0.686499  
                    ломтик июльского неба                               14.416476  
                    плюмбус                                              5.377574  
                    подписка на suppi-блог                               2.517162  
                    статуэтка Ленина                                     4.004577  
гиперборея          автограф Стаса Барецкого                            21.068702  
                    билет в Израиль                                     59.389313  
                    зеленая пластинка                                   -2.366412  
                    ломтик июльского неба                                6.106870  
                    плюмбус                                              8.702290  
                    подписка на suppi-блог                               7.099237  
остров невезения    автограф Стаса Барецкого                            14.372716  
                    билет в Израиль                                     12.119367  
                    зеленая пластинка                                   -0.182704  
                    ломтик июльского неба                               26.918392  
                    плюмбус                                             23.325213  
                    статуэтка Ленина                   

### 5. Сортировка столбца  'percent_profit_product_of_warehouse' полученной таблицы по убыванию, подсчёт накопленного процента и добавление нового столбца 'accumulated_percent_profit_product_of_warehouse' с полученным значением. 

In [30]:
final_df.sort_values(
    by="percent_profit_product_of_warehouse", ascending=False, inplace=True
)

In [31]:
final_df["accumulated_percent_profit_product_of_warehouse"] = final_df[
    "percent_profit_product_of_warehouse"
].cumsum()

In [32]:
display(final_df.head(10))

quantity  profit  \
warehouse_name      product                                      
Мордор              билет в Израиль                  6  5930.0   
отель Лето          билет в Израиль                 10  9525.0   
гиперборея          билет в Израиль                  8  7780.0   
хутор близ Диканьки автограф Стаса Барецкого        11  6225.0   
                    билет в Израиль                  6  5805.0   
остров невезения    ломтик июльского неба            5  2210.0   
                    статуэтка Ленина                10  1925.0   
                    плюмбус                          8  1915.0   
гиперборея          автограф Стаса Барецкого         5  2760.0   
отель Лето          ломтик июльского неба            8  3300.0   

                                              percent_profit_product_of_warehouse  \
warehouse_name      product                                                         
Мордор              билет в Израиль                                     67.848970   
отель Лето          билет в Израиль                                     60.056747   
гиперборея          билет в Израиль                                     59.389313   
хутор близ Диканьки автограф Стаса Барецкого                            33.342260   
                    билет в Израиль                                     31.092662   
остров невезения    ломтик июльского неба                               26.918392   
                    статуэтка Ленина                                    23.447016   
                    плюмбус                                             23.325213   
гиперборея          автограф Стаса Барецкого                            21.068702   
отель Лето          ломтик июльского неба                               20.807062   

                                              accumulated_percent_profit_product_of_warehouse  
warehouse_name      product                                                                    
Мордор              билет в Израиль                                                 67.848970  
отель Лето          билет в Израиль                                                127.905717  
гиперборея          билет в Израиль                                                187.295030  
хутор близ Диканьки автограф Стаса Барецкого                                       220.637290  
                    билет в Израиль                                                251.729952  
остров невезения    ломтик июльского неба                                          278.648344  
                    статуэтка Ленина                                               302.095360  
                    плюмбус                                                        325.420573  
гиперборея          автограф Стаса Барецкого                                       346.489276  
отель Лето          ломтик июльского неба                                          367.296337

### 6. Создание нового столбца 'category' с присвоением A,B,C - категорий на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'):  

**Если значение накопленного процента меньше или равно 70, то категория A.  
 Если от 70 до 90 (включая 90), то категория Б.  
 Остальное - категория C.**


In [33]:
bins = [0, 70, 90, np.inf]

labels = ["A", "B", "C"]

final_df["category"] = pd.cut(
    final_df["accumulated_percent_profit_product_of_warehouse"],
    bins=bins,
    labels=labels,
)

In [34]:
display(final_df)

quantity  profit  \
warehouse_name      product                                      
Мордор              билет в Израиль                  6  5930.0   
отель Лето          билет в Израиль                 10  9525.0   
гиперборея          билет в Израиль                  8  7780.0   
хутор близ Диканьки автограф Стаса Барецкого        11  6225.0   
                    билет в Израиль                  6  5805.0   
остров невезения    ломтик июльского неба            5  2210.0   
                    статуэтка Ленина                10  1925.0   
                    плюмбус                          8  1915.0   
гиперборея          автограф Стаса Барецкого         5  2760.0   
отель Лето          ломтик июльского неба            8  3300.0   
хутор близ Диканьки плюмбус                         16  3445.0   
Мордор              ломтик июльского неба            3  1260.0   
остров невезения    автограф Стаса Барецкого         2  1180.0   
отель Лето          автограф Стаса Барецкого         4  2250.0   
остров невезения    билет в Израиль                  1   995.0   
хутор близ Диканьки статуэтка Ленина                10  1640.0   
гиперборея          плюмбус                          6  1140.0   
                    подписка на suppi-блог           9   930.0   
хутор близ Диканьки ломтик июльского неба            3  1305.0   
Мордор              автограф Стаса Барецкого         1   570.0   
гиперборея          ломтик июльского неба            2   800.0   
Мордор              плюмбус                          2   470.0   
                    статуэтка Ленина                 2   350.0   
отель Лето          статуэтка Ленина                 3   475.0   
                    плюмбус                          2   450.0   
Мордор              подписка на suppi-блог           2   220.0   
хутор близ Диканьки подписка на suppi-блог           3   360.0   
отель Лето          подписка на suppi-блог           2   175.0   
остров невезения    зеленая пластинка                4   -15.0   
хутор близ Диканьки зеленая пластинка                4  -110.0   
Мордор              зеленая пластинка               10   -60.0   
отель Лето          зеленая пластинка                6  -315.0   
гиперборея          зеленая пластинка                7  -310.0   

                                              percent_profit_product_of_warehouse  \
warehouse_name      product                                                         
Мордор              билет в Израиль                                     67.848970   
отель Лето          билет в Израиль                                     60.056747   
гиперборея          билет в Израиль                                     59.389313   
хутор близ Диканьки автограф Стаса Барецкого                            33.342260   
                    билет в Израиль                                     31.092662   
остров невезения    ломтик июльского неба                               26.918392   
                    статуэтка Ленина                                    23.447016   
                    плюмбус                                             23.325213   
гиперборея          автограф Стаса Барецкого                            21.068702   
отель Лето          ломтик июльского неба                               20.807062   
хутор близ Диканьки плюмбус                                             18.452062   
Мордор              ломтик июльского неба                               14.416476   
остров невезения    автограф Стаса Барецкого                            14.372716   
отель Лето          автограф Стаса Барецкого                            14.186633   
остров невезения    билет в Израиль                                     12.119367   
хутор близ Диканьки статуэтка Ленина                                     8.784146   
гиперборея          плюмбус                                              8.702290   
                    подписка на suppi-блог                               7.099237   
хутор близ Диканьки ломтик июльског